In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

box_DIR = "../data/misc/"
fig_DIR = "../figs/analysis/"

In [ ]:
from nba_api.stats.endpoints import leaguedashptdefend

In [ ]:
dcat = ["Overall","3 Pointers","2 Pointers","Less Than 6Ft","Less Than 10Ft","Greater Than 15Ft"]
dfc=[]
for year in range(2013,2024):
    dfa = []
    for i in range(len(dcat)):

        season_str = get_ss(year)
        
        stats = leaguedashptdefend.LeagueDashPtDefend(
            league_id="00",
            season=season_str,
            per_mode_simple="Totals",
            season_type_all_star="Regular Season",
            defense_category=dcat[i],

        )
        df = stats.get_data_frames()[0]
        df = df.drop(columns=['PLAYER_LAST_TEAM_ID',
        'PLAYER_LAST_TEAM_ABBREVIATION', 'PLAYER_POSITION', 'AGE', 'GP', 'G',])
        # df["season"]=year+1
        dfa.append(df)
        time.sleep(0.6)
    dfa[0].columns = ['pID', 'NAME', 'FREQ', 'D_FGM', 'D_FGA',
        'D_FG_PCT', 'N_FG_PCT', 'DIFF']
    for i in range(1,len(dcat)):
        dfa[i].columns = ['pID', 'NAME', f'FREQ_{i}', f'D_FGM_{i}', f'D_FGA_{i}',
        f'D_FG_PCT_{i}', f'N_FG_PCT_{i}', f'DIFF_{i}']
    dfb = pd.merge(dfa[0],dfa[1],on=["pID","NAME"])
    for i in range(2,len(dcat)):
        dfb = pd.merge(dfb,dfa[i])
    cols = ['FREQ','D_FGM','D_FGA']
    i1 = 1
    i2 = 5
    i3 = 6
    for col in cols:
        dfb[f"{col}_{i3}"]=dfb[f"{col}_{i2}"]-dfb[f"{col}_{i1}"]
    dfb[f"D_FG_PCT_{i3}"]=dfb[f"D_FGM_{i3}"]/dfb[f"D_FGA_{i3}"]
    dfb[f"D_FG_PCT_{i3}"]=dfb[f"D_FG_PCT_{i3}"].round(3)
    cols = ['FREQ','D_FGM','D_FGA']
    i1 = 3
    i2 = 4
    i3 = 7
    for col in cols:
        dfb[f"{col}_{i3}"]=dfb[f"{col}_{i2}"]-dfb[f"{col}_{i1}"]
    dfb[f"D_FG_PCT_{i3}"]=dfb[f"D_FGM_{i3}"]/dfb[f"D_FGA_{i3}"]
    dfb[f"D_FG_PCT_{i3}"]=dfb[f"D_FG_PCT_{i3}"].round(3)
    cols = ['FREQ','D_FGM','D_FGA']
    i11 = 4
    i12 = 6
    i2 = 2
    i3 = 8
    for col in cols:
        dfb[f"{col}_{i3}"]=dfb[f"{col}_{i2}"]-dfb[f"{col}_{i11}"]-dfb[f"{col}_{i12}"]
    dfb[f"D_FG_PCT_{i3}"]=dfb[f"D_FGM_{i3}"]/dfb[f"D_FGA_{i3}"]
    dfb[f"D_FG_PCT_{i3}"]=dfb[f"D_FG_PCT_{i3}"].round(3)
    dfb["season"]=year+1
    dfc.append(dfb)

In [ ]:
dfd = []
for ii in range(len(dfc)-1):
    df1 = pd.merge(dfc[ii],dfc[ii+1],on=['pID', 'NAME'])
    df1 = df1.fillna(0)
    dfd.append(df1)
dfe = pd.concat(dfd)

0: Overall  
1: 3Pt  
2: 2Pt  
3: Less Than 6Ft  
4: Less Than 10Ft  
5: Greater Than 15Ft   
6: 15Ft-3Pt  
7: 6Ft-10Ft  
8: 10Ft-15Ft

In [ ]:
for j in range(len(dfd)):
    data = dfd[j]
    # for k in range(1,9):
    for k in [3]:
        var = f"D_FG_PCT_{k}"
        data1 = data.query(f"D_FGA_{k}_x>400 & D_FGA_{k}_y>400")
        var1 = var + "_x"
        var2 = var + "_y"
        slope, intercept, r, p, sterr = scipy.stats.linregress(x=data1[var1], y=data1[var2])
        r2 = r**2
        print(f"{2013+j}:{round(r2,3)}")

In [ ]:
# dfd = dfc.query("FG2A_x> 100 & FG2A_y> 100")
dfd = dfc.query("FGA_LT_06_x> 500 & FGA_LT_06_x> 500")
# dfd = dfc.query("FGA_GT_15_x> 100 & FGA_GT_15_x> 100")
# dfd = dfc.query("FG3A_x> 100 & FG3A_y> 100")
len(dfd)

In [ ]:
var = "LT_06_PCT"
var1 = var + "_x"
var2 = var + "_y"
slope, intercept, r, p, sterr = scipy.stats.linregress(x=dfd[var1], y=dfd[var2])
r2 = r**2
fig, ax = plt.subplots(1, 1)
fig = sns.regplot(
    x=var1,
    y=var2,
    data=dfd,
    color="black",
    scatter_kws={"color": "tab:blue"},
    ax=ax,
)
ax.text(0.05, 0.9, r"$r^2=$" + f"{round(r2,4)}", transform=ax.transAxes)
ax.set_title("Defended Less Than 6ft Year vs Year+1")
ax.set_xlabel("Year")
ax.set_ylabel("Year + 1")
plt.savefig(fig_DIR + "def_2pt_l6_1.png")
plt.show()

In [ ]:
var = "PLUSMINUS"
var1 = var + "_x"
var2 = var + "_y"
slope, intercept, r, p, sterr = scipy.stats.linregress(x=dfd[var1], y=dfd[var2])
r2 = r**2
fig, ax = plt.subplots(1, 1)
fig = sns.regplot(
    x=var1,
    y=var2,
    data=dfd,
    color="black",
    scatter_kws={"color": "tab:blue"},
    ax=ax,
)
ax.text(0.05, 0.9, r"$r^2=$" + f"{round(r2,4)}", transform=ax.transAxes)
ax.set_title("Defended Less Than 6ft Diff Year vs Year+1")
ax.set_xlabel("Year")
ax.set_ylabel("Year + 1")
plt.savefig(fig_DIR + "def_2pt_l6_diff_1.png")
plt.show()

In [ ]:
fgdfg

In [ ]:
i = 1
dfa = []
dcat = ["Overall","3 Pointers","2 Pointers","Less Than 6Ft","Less Than 10Ft","Greater Than 15Ft"]
for year in range(2013,2024):
    season_str = get_ss(year)
    
    stats = leaguedashptdefend.LeagueDashPtDefend(
        league_id="00",
        season=season_str,
        per_mode_simple="Totals",
        season_type_all_star="Regular Season",
        defense_category=dcat[i],

    )
    df = stats.get_data_frames()[0]
    df["season"]=year+1
    dfa.append(df)
    time.sleep(0.6)

In [ ]:
dfb = []
for ii in range(len(dfa)-1):
    df1 = pd.merge(dfa[ii],dfa[ii+1],on=["CLOSE_DEF_PERSON_ID","PLAYER_NAME"])
    dfb.append(df1)
dfc = pd.concat(dfb)
dfc = dfc.drop(columns=['PLAYER_LAST_TEAM_ID_x',
       'PLAYER_LAST_TEAM_ABBREVIATION_x', 'PLAYER_POSITION_x', 'AGE_x', 'GP_x',
       'G_x','PLAYER_LAST_TEAM_ID_y',
       'PLAYER_LAST_TEAM_ABBREVIATION_y', 'PLAYER_POSITION_y', 'AGE_y', 'GP_y',
       'G_y',])


In [ ]:
dfc.columns

In [ ]:
# dfd = dfc.query("FG2A_x> 100 & FG2A_y> 100")
# dfd = dfc.query("FGA_LT_06_x> 500 & FGA_LT_06_x> 500")
# dfd = dfc.query("FGA_GT_15_x> 100 & FGA_GT_15_x> 100")
dfd = dfc.query("FG3A_x> 300 & FG3A_y> 300")
len(dfd)

In [ ]:
var = "FG3_PCT"
var1 = var + "_x"
var2 = var + "_y"
slope, intercept, r, p, sterr = scipy.stats.linregress(x=dfd[var1], y=dfd[var2])
r2 = r**2
fig, ax = plt.subplots(1, 1)
fig = sns.regplot(
    x=var1,
    y=var2,
    data=dfd,
    color="black",
    scatter_kws={"color": "tab:blue"},
    ax=ax,
)
ax.text(0.05, 0.9, r"$r^2=$" + f"{round(r2,4)}", transform=ax.transAxes)
ax.set_title("Defended 3pt Year vs Year+1")
ax.set_xlabel("Year")
ax.set_ylabel("Year + 1")
plt.savefig(fig_DIR + "def_3pt_1.png")
plt.show()

In [ ]:
var = "PLUSMINUS"
var1 = var + "_x"
var2 = var + "_y"
slope, intercept, r, p, sterr = scipy.stats.linregress(x=dfd[var1], y=dfd[var2])
r2 = r**2
fig, ax = plt.subplots(1, 1)
fig = sns.regplot(
    x=var1,
    y=var2,
    data=dfd,
    color="black",
    scatter_kws={"color": "tab:blue"},
    ax=ax,
)
ax.text(0.05, 0.9, r"$r^2=$" + f"{round(r2,4)}", transform=ax.transAxes)
ax.set_title("Defended 3pt Diff Year vs Year+1")
ax.set_xlabel("Year")
ax.set_ylabel("Year + 1")
plt.savefig(fig_DIR + "def_3pt_diff_1.png")
plt.show()